In [25]:
import numpy as np
import cv2

nx, ny = 5, 8
objpoints = []

def getInfo(video):
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frame_count / fps
    return int(fps), int(frame_count), int(duration)

def binarizar(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(frame, 9)
    ret, binarizado = cv2.threshold(blur, 4, 255, cv2.THRESH_BINARY)
    binarizado = cv2.dilate(binarizado, cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (3,3)
    ), iterations = 2)
    return binarizado

def getCorners(frame, nx, ny):
    ret, corners = cv2.findChessboardCorners(frame, (nx, ny))
    # print(corners)
    return ret, corners

# def getPlanoDeFundo(frame):
#     fgmask = cv2.createBackgroundSubtractorMOG2().apply(frame)
#     fgmask = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
#     cv2.imshow('fundo', fgmask)
#     return fgmask

def desenharPontos(frame_inicial, ret, corners, nx, ny, objpoints):
    objp = np.zeros((nx*ny,2), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
    if ret:
        pontos = np.int0(corners)
        for c in pontos:
            x,y = c.ravel()
            cv2.circle(frame_inicial, (x,y), 3, (0,0,255), -1)
        objpoints.append(objp)
        cv2.imshow('pontos', frame_inicial)

def getFrame(video, posicao):
    video.set(cv2.CAP_PROP_POS_FRAMES, posicao)
    res, frame = video.read()
    return res, frame

video = cv2.VideoCapture('xadrez.mp4')
fps, frame_count, duration = getInfo(video)

# print(objpoints, corners)

# Calcular aqui a homografia
# h, mask = cv2.findHomography(corners, objpoints, cv2.RANSAC)
# print(h)

# frame com o tabuleiro
_, frame_inicial = getFrame(video, 80)
frame_inicial = cv2.cvtColor(frame_inicial, cv2.COLOR_BGR2HSV)
# frame com o fundo
_, frame_fundo = getFrame(video, 150)
# Seta para o momento da bolinha (não precisamos percorrer o vídeo inteiro)
video.set(cv2.CAP_PROP_POS_FRAMES, 220)
# Converte para grayscale
gray = cv2.cvtColor(frame_inicial, cv2.COLOR_BGR2GRAY)
# Peg os cantos (nx, ny)
ret, corners = getCorners(gray, nx, ny)
# Desenha os pontos no tabuleiro (opcional)
desenharPontos(frame_inicial, ret, corners, nx, ny, objpoints)

# Loop no vídeo (frame a frame)
while video.isOpened():
    ok, frame = video.read()
    if frame is None:
        break
    
    bolinha = cv2.subtract(frame, frame_fundo)
    binarizada = binarizar(bolinha)
    # # cantos = np.int0(cantos)

    # for i in cantos:
    #     a = []
    #     b = []

    #     x, y = i.ravel()
    #     cv2.circle(binarizada, (x,y), 7, (0,255,0), -1)
    #     a.append(x)
    #     b.append(y)
    
    # print(a, b)
    
    # encontrar o menor ponto Y (branco)
    # desenhar no frame bolinha
    # multiplicação de H pelo menor ponto Y encontrado
    # matplotlib para plotar os pontos

    
    
    # cv2.imshow('frame', frame)
    # cv2.imshow('fundo', fundo)
    cv2.imshow('bolinha', bolinha)
    cv2.imshow('binarizado', binarizada)
    
    
    key = cv2.waitKey(fps) & 0xff
    if key == ord('q'):
        break

video.release()
cv2.destroyAllWindows() 